<a href="https://colab.research.google.com/github/DanilaK10/Heart-Disease-Prediction-dataset/blob/main/RandomForestGridCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/DanilaK10/Heart-Disease-Prediction-dataset/main/Heart%20Disease%20Dataset.csv")
df

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve

import warnings
warnings.filterwarnings("ignore")

#renaming columns - Kaggle - 3.2
df.columns = ["Age", "Sex", "Chest_pain_type", "Resting_bp", "Cholesterol", "Fasting_bs", "Resting_ecg", "Max_heart_rate", "Exercise_induced_angina", "ST_depression", "ST_slope", "Num_major_vessels", "Thallium_test", "Condition"]
df

y = df["Condition"] # - y is the condition values from the dataset
x = df.drop("Condition", axis = 1) # - x is the dataset with condition removed
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=23)
x_test

def get_model_accuracy(model, X_test, y_test):
    #Return the mean accuracy of model on X_test and y_test
    model_acc = model.score(X_test, y_test)
    return model_acc


def get_normalization(X):
    scaler = MinMaxScaler()
    X_normalized = scaler.fit_transform(X)
    return X_normalized


x_train = get_normalization(x_train);
x_test = get_normalization(x_test);


rf_model = RandomForestClassifier()
rf_model = rf_model.fit(x_train, y_train)

# Accuracy score for Random Forest
rf_acc = get_model_accuracy(rf_model, x_test, y_test)
print(f'RandomForestClassifier Accuracy: {rf_acc:.4}')
print()

# Predict class for X_test
y_pred_rf = rf_model.predict(x_test)

# Classification Report of Random Forest model
print(classification_report(y_pred_rf, y_test))




def get_best_parameters_GridSearchCV(model, params, X_train, y_train):
    clf = GridSearchCV(model, params, cv=5)
    clf.fit(X_train, y_train)

    best_params = clf.best_params_
    print(f'Best Parameters in {model}: {best_params}')
    print()

    best_estimator = clf.best_estimator_
    return best_estimator


params_rf = {'max_depth': [2, 3, 4, 5],
               'max_features': ['auto', 'sqrt', 'log2'],
               'n_estimators':[0, 10, 50],
               'random_state': [0, 10, 42]}
# Build and fit Random Forest model with the best hyperparameters
rf_gscv = get_best_parameters_GridSearchCV(rf_model, params_rf, x_train, y_train)
rf_gscv = rf_gscv.fit(x_train, y_train)

# Accuracy score for rf_gscv
rf_gscv_acc = get_model_accuracy(rf_gscv, x_test, y_test)
print(f'Random Forest with GridSearchCV Accuracy: {rf_gscv_acc:.4}')
print()

# Make prediction on test dataset
y_pred_rf_gscv = rf_gscv.predict(x_test)

# Classification Report of grid_rf_model
print(classification_report(y_pred_rf_gscv, y_test))

grid = GridSearchCV(rf_model, params_rf, cv = 5, scoring = "accuracy")
grid.fit(x_train, y_train)
pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']]

RandomForestClassifier Accuracy: 0.8033

              precision    recall  f1-score   support

           0       0.80      0.80      0.80        30
           1       0.81      0.81      0.81        31

    accuracy                           0.80        61
   macro avg       0.80      0.80      0.80        61
weighted avg       0.80      0.80      0.80        61

Best Parameters in RandomForestClassifier(): {'max_depth': 5, 'max_features': 'auto', 'n_estimators': 50, 'random_state': 42}

Random Forest with GridSearchCV Accuracy: 0.8689

              precision    recall  f1-score   support

           0       0.80      0.92      0.86        26
           1       0.94      0.83      0.88        35

    accuracy                           0.87        61
   macro avg       0.87      0.88      0.87        61
weighted avg       0.88      0.87      0.87        61



,mean_test_score,std_test_score,params
0,NaN,NaN,"{'max_depth': 2, 'max_features': 'auto', 'n_es..."
1,NaN,NaN,"{'max_depth': 2, 'max_features': 'auto', 'n_es..."
2,NaN,NaN,"{'max_depth': 2, 'max_features': 'auto', 'n_es..."
3,0.776786,0.051545,"{'max_depth': 2, 'max_features': 'auto', 'n_es..."
4,0.818282,0.043499,"{'max_depth': 2, 'max_features': 'auto', 'n_es..."
...,...,...,...
103,0.789116,0.040486,"{'max_depth': 5, 'max_features': 'log2', 'n_es..."
104,0.838946,0.029945,"{'max_depth': 5, 'max_features': 'log2', 'n_es..."
105,0.830782,0.043032,"{'max_depth': 5, 'max_features': 'log2', 'n_es..."
106,0.826531,0.027511,"{'max_depth': 5, 'max_features': 'log2', 'n_es..."


Most Relevant Factors for Heart Disease (Descending Order):
Chest_pain_type: 0.1632
Max_heart_rate: 0.1128
Num_major_vessels: 0.1110
Thallium_test: 0.1021
Age: 0.0931
ST_depression: 0.0921
Resting_bp: 0.0869
Cholesterol: 0.0839
Exercise_induced_angina: 0.0664
ST_slope: 0.0316
Sex: 0.0272
Resting_ecg: 0.0198
Fasting_bs: 0.0100
RandomForestClassifier Accuracy: 0.8361